In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(threshold=10, edgeitems=5)

定义漫步次数、步数、维度

In [ ]:
N = 5000
nsteps = 1000
dim = 3

In [ ]:
walks = np.arange(N)
steps = np.arange(nsteps)

对每次漫步的每一步，随机一个移动维度和移动距离

In [ ]:
rng = np.random.default_rng()
dim_indices = rng.integers(0, dim, size=(N, nsteps))
distances = np.random.choice((1, -1), size=(N, nsteps))
dim_indices
distances

初始化位移向量，并根据选取的维度用移动距离赋值

In [ ]:
# %%timeit
walk_indices = np.repeat(walks, nsteps).reshape(N, nsteps)
step_indices = np.tile(steps, N).reshape(N, nsteps)
displacement_vectors = np.zeros((N, nsteps, dim))
displacement_vectors[walk_indices, step_indices, dim_indices] = distances
walk_indices
step_indices
displacement_vectors

可能的另一种实现方式，但似乎无法输出结果

In [ ]:
# # %%timeit
# displacement_vectors = np.zeros((N, nsteps, dim))
# displacement_vectors[:, :, dim_indices] = distances
# displacement_vectors

累加位移向量，获得每次每步的位置

In [ ]:
pos_without_start = displacement_vectors.cumsum(axis=1)
pos_without_start

在每次漫步的第一步插入起始位置

In [ ]:
start_pos = np.zeros((N, 1, dim))
pos = np.concatenate((start_pos, pos_without_start), axis=1)
pos

计算每次每步的位置与原点的欧氏距离

In [ ]:
euclidean_distance = np.linalg.norm(pos, ord=2, axis=2)
euclidean_distance
euclidean_distance.shape

计算欧氏距离超过30的漫步的次数

In [ ]:
arrive30 = (euclidean_distance >= 30).any(axis=1)
arrive30
arrive30.sum()

计算抵达目标距离的漫步的平均最小抵达时间

In [ ]:
arrive30_times = (euclidean_distance[arrive30] >= 30).argmax(axis=1)
(euclidean_distance[arrive30] >= 30).shape
arrive30_times
arrive30_times.mean()

随机选择10次漫步，并绘制三维路线图

In [ ]:
selected_indices = np.random.choice(N, size=10, replace=False)
selected_walks = pos[selected_indices]
selected_indices
selected_walks

In [ ]:
fig = plt.figure(figsize=(14, 12), dpi=300)
ax = fig.add_subplot(111, projection='3d')

for walk in selected_walks:
    x = walk[:, 0]  # x坐标序列
    y = walk[:, 1]  # y坐标序列
    z = walk[:, 2]  # z坐标序列
    ax.plot(x, y, z, alpha=0.3, lw=0.8, color='blue')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('三维随机漫步（选取10次）')
ax.grid(True, alpha=0.2)  # 浅色网格线
ax.view_init(elev=30, azim=45)  # 调整视角（仰角30°，方位角45°）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False

# plt.savefig('3d_walk_highres.png', dpi=300, bbox_inches='tight') # 用于将生成的图片保存为文件
plt.show()